Columns:

,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature

In [23]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder

In [24]:
song_data = pd.read_csv('data/song_data.csv')
# Remove key, mode, time_signature, and duration_ms
song_data = song_data.drop(['key', 'mode', 'time_signature', 'duration_ms'], axis=1)

## Implement KNN

#### Feature scaling

In [37]:
# Scale numerical features
numerical_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']
X = song_data[numerical_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [39]:
# Genre encoding
genres = song_data[['genre']]
encoder = OneHotEncoder(sparse=False)
genres_encoded = encoder.fit_transform(genres)#.toarray()

c:\Users\P_Osk\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


#### Combine genres and numerical features and fit

In [40]:
genre_weight = 20 # The higher the weight, the more important the genre is in the recommendation

X_combined = np.concatenate((X_scaled, genres_encoded * genre_weight), axis=1)

knn = NearestNeighbors(n_neighbors=10, metric='euclidean')
knn.fit(X_combined)

NearestNeighbors(metric='euclidean', n_neighbors=10)

## Song Query

In [41]:
# Define song and artist which we want to find similar songs for
song_name = 'What Once Was'
artist_name = 'Her\'s'

In [45]:
# Find the song in the dataset
song_query = song_data[(song_data['track_name'].str.lower() == song_name.lower()) & (song_data['artist_name'].str.lower() == artist_name.lower())]

if not song_query.empty:
    # Scale numerical features of the query
    song_numerical_features = scaler.transform(song_query[numerical_features])
    
    # Encode genre of the query
    song_genre_encoded = encoder.transform(song_query[['genre']])#.toarray()
    
    # Combine scaled numerical features and encoded genre for the query
    song_combined_features = np.hstack((song_numerical_features, song_genre_encoded * 20))
    
    # Use KNN to find the nearest neighbors
    distances, indices = knn.kneighbors(song_combined_features)
    
    print('Closest songs to |', song_name, '-', artist_name, '| are...')
    for i in range(1, 6):
        index = indices[0][i]
        print('----------')
        print(song_data.iloc[index][['track_name', 'artist_name']])
        
else:
    print("No song found. Check the song and artist names.")

Closest songs to | What Once Was - Her's | are...
----------
track_name         Rosary
artist_name    Inner Wave
Name: 131030, dtype: object
----------
track_name              Lupa
artist_name    King Stingray
Name: 553167, dtype: object
----------
track_name     Stupid Decisions
artist_name              FIDLAR
Name: 180455, dtype: object
----------
track_name         2:00 AM
artist_name    Los Shadows
Name: 389334, dtype: object
----------
track_name     Follow The Sun
artist_name    Royal Republic
Name: 236101, dtype: object


## Playlist testing

In [110]:
'''
playlist_test = [#('What Once Was', 'Her\'s'), 
                 ('Lauren', 'Men I Trust'),#,
                 #('Pink + White', 'Frank Ocean'), 
                 #('After The Earthquake', 'Alvvays'),
                 #('North', 'Clairo'),
                 #('Sunflower', 'Rex Orange County'),
                 ('Wet Dream', 'Wet Leg'),
                 ('Freaking Out the Neighborhood', 'Mac DeMarco')]#,
                 #('Lovers Rock', 'TV Girl'),
                 #('Right Side of My Neck', 'Faye Webster'),
                 #('Summertime Magic', 'Childish Gambino'),
                 #('Be Sweet', 'Japanese Breakfast'),
                 #('Where U Goin\' Tonight?', 'Mac Ayres'),
                 #('Just A Stranger (feat. Steve Lacy)', 'Kali Uchis')]#,
                 #('Useless', 'Omar Apollo')]
'''

playlist_test2 = [('Easy Lover', 'Philip Bailey'),
                  ('505', 'Arctic Monkeys'),
                  ('Electric Feel', 'MGMT'),
                  ('Everywhere - 2018 Remaster', 'Fleetwood Mac'),
                  ('Blue Monday \'88 - 2015 Remaster', 'New Order')] 


playlist_test3 = [('Material Girl', 'Madonna'),
                  ('Total Eclipse of the Heart', 'Bonnie Tyler'),
                  ('I Wanna Dance with Somebody (Who Loves Me)', 'Whitney Houston'),
                  ('Girls Just Want to Have Fun', 'Cyndi Lauper'),
                  ('Take On Me', 'a-ha')]

In [111]:

playlist_numerical_features = []
playlist_genre_encoded = []

for song_name, artist_name in playlist_test3:
    song_query = song_data[(song_data['track_name'].str.lower() == song_name.lower()) & (song_data['artist_name'].str.lower() == artist_name.lower())]

    if not song_query.empty:
        # Scale numerical features of the query
        song_numerical_features = scaler.transform(song_query[numerical_features])
        playlist_numerical_features.append(song_numerical_features)
        
        # Encode genre of the query
        song_genre_encoded = encoder.transform(song_query[['genre']])#.toarray()
        playlist_genre_encoded.append(song_genre_encoded)
    else:
        print(f'No song found for {song_name} by {artist_name}. Check the song and artist names.')

# Aggregate features of the playlist
if playlist_numerical_features and playlist_genre_encoded:

    average_numerical_features = np.mean(np.vstack(playlist_numerical_features), axis=0)
    average_genre_encoded = np.mean(np.vstack(playlist_genre_encoded), axis=0)

    # Determine variance across the playlist for each numerical feature to use as a simple importance metric
    variances = np.var(np.vstack(playlist_numerical_features), axis=0)
    importance_weights = 1 / (variances + 1e-6)  
    normalized_importance_weights = importance_weights / importance_weights.max()

    # Apply weights to the aggregated playlist features
    weighted_playlist_features = average_numerical_features * normalized_importance_weights

    # Apply the same weighting to the full dataset numerical features before fitting KNN
    weighted_numerical_features = X_scaled * normalized_importance_weights

    knn = NearestNeighbors(n_neighbors=10, metric='euclidean')
    knn.fit(X_combined)

    # Re-combine weighted numerical features with encoded categorical for the full dataset
    playlist_combined_features = np.hstack([weighted_playlist_features, average_genre_encoded])

    distances, indices = knn.kneighbors([playlist_combined_features])

    print('Closest songs to the playlist are...')
    for i in range(5):
        index = indices[0][i]
        print('----------')
        print(song_data.iloc[index][['track_name', 'artist_name']])
else:
    print('The playlist is empty or no songs were found.')


Closest songs to the playlist are...
----------
track_name     Forever Yours - Avicii Tribute
artist_name                              Kygo
Name: 433582, dtype: object
----------
track_name     Player's Prayer
artist_name              Lloyd
Name: 935675, dtype: object
----------
track_name     Still Falling For You - Jonas Blue Remix
artist_name                              Ellie Goulding
Name: 224778, dtype: object
----------
track_name     I Love You Always Forever
artist_name                    Betty Who
Name: 224622, dtype: object
----------
track_name     Slow Wine Bass Line (feat. Teddy Riley)
artist_name                                      Lloyd
Name: 433944, dtype: object
